##Vectorized PandasUDF + Keras for Inference

One of the exciting things we can do with Arrow/Vectorized PandasUDF is efficiently integrate Spark with numeric or even GPU code that was *not* designed with Spark in mind.

For example, we might have a model that we've build with Keras+TensorFlow -- without Spark in the picture at all -- and then efficiently use that model to perform inference on big datasets with Spark.

In this module, we'll do just that: we'll train a simple neural network using Keras, save it to disk, and then use it from Spark via PandasUDF.

We'll start by using Pandas to read our Diamonds dataset, and to save time (and featurization) we'll just use the 6 continuous variables in our model to predict price.

In [3]:
%%sh 

wget https://materials.s3.amazonaws.com/res/data/diamonds.csv -O /tmp/diamonds.csv

--2019-04-30 18:51:03-- https://materials.s3.amazonaws.com/res/data/diamonds.csv
Resolving materials.s3.amazonaws.com (materials.s3.amazonaws.com)... 52.218.225.115
Connecting to materials.s3.amazonaws.com (materials.s3.amazonaws.com)|52.218.225.115|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3192559 (3.0M) [application/octet-stream]
Saving to: ‘/tmp/diamonds.csv’

 0K .......... .......... .......... .......... .......... 1% 23.2M 0s
 50K .......... .......... .......... .......... .......... 3% 27.5M 0s
 100K .......... .......... .......... .......... .......... 4% 36.9M 0s
 150K .......... .......... .......... .......... .......... 6% 21.3M 0s
 200K .......... .......... .......... .......... .......... 8% 44.7M 0s
 250K .......... .......... .......... .......... .......... 9% 35.3M 0s
 300K .......... .......... .......... .......... .......... 11% 26.8M 0s
 350K .......... .......... .......... .......... .......... 12% 46.6M 0s
 400K .......... .......... .......... .......... .......... 14% 46.6M 0s
 450K .......... .......... .......... .......... .......... 16% 41.2M 0s
 500K .......... .......... .......... .......... .......... 17% 40.0M 0s
 550K .......... .......... .......... .......... .......... 19% 37.1M 0s
 600K .......... .......... .......... .......... .......... 20% 46.0M 0s
 650K .......... .......... .......... .......... .......... 22% 49.6M 0s
 700K .......... .......... .......... .......... .......... 24% 48.5M 0s
 750K .......... .......... .......... .......... .......... 25% 63.9M 0s
 800K .......... .......... .......... .......... .......... 27% 58.2M 0s
 850K .......... .......... .......... .......... .......... 28% 82.6M 0s
 900K .......... .......... .......... .......... .......... 30% 67.8M 0s
 950K .......... .......... .......... .......... .......... 32% 71.6M 0s
 1000K .......... .......... .......... .......... .......... 33% 65.3M 0s
 1050K .......... .......... .......... .......... .......... 35% 68.1M 0s
 1100K .......... .......... .......... .......... .......... 36% 63.5M 0s
 1150K .......... .......... .......... .......... .......... 38% 108M 0s
 1200K .......... .......... .......... .......... .......... 40% 53.3M 0s
 1250K .......... .......... .......... .......... .......... 41% 116M 0s
 1300K .......... .......... .......... .......... .......... 43% 40.4M 0s
 1350K .......... .......... .......... .......... .......... 44% 105M 0s
 1400K .......... .......... .......... .......... .......... 46% 61.0M 0s
 1450K .......... .......... .......... .......... .......... 48% 107M 0s
 1500K .......... .......... .......... .......... .......... 49% 55.6M 0s
 1550K .......... .......... .......... .......... .......... 51% 132M 0s
 1600K .......... .......... .......... .......... .......... 52% 76.7M 0s
 1650K .......... .......... .......... .......... .......... 54% 114M 0s
 1700K .......... .......... .......... .......... .......... 56% 105M 0s
 1750K .......... .......... .......... .......... .......... 57% 89.3M 0s
 1800K .......... .......... .......... .......... .......... 59% 98.0M 0s
 1850K .......... .......... .......... .......... .......... 60% 76.8M 0s
 1900K .......... .......... .......... .......... .......... 62% 87.3M 0s
 1950K .......... .......... .......... .......... .......... 64% 86.4M 0s
 2000K .......... .......... .......... .......... .......... 65% 81.5M 0s
 2050K .......... .......... .......... .......... .......... 67% 87.3M 0s
 2100K .......... .......... .......... .......... .......... 68% 108M 0s
 2150K .......... .......... .......... .......... .......... 70% 80.7M 0s
 2200K .......... .......... .......... .......... .......... 72% 70.2M 0s
 2250K .......... .......... .......... .......... .......... 73% 141M 0s
 2300K .......... .......... .......... .......... .......... 75% 127M 0s
 2350K .......... .......... .......... .......... .......... 76% 87.9M 0s
 2400K .......... .......... .......... .

In [4]:
import pandas as pd
import IPython.display as disp

input_file = "/tmp/diamonds.csv"

df = pd.read_csv(input_file, header = 0)
df.drop(df.columns[0], axis=1, inplace=True)
df.drop(df.columns[1:4], axis=1, inplace=True)
disp.display(df)

carat depth table price x y z
0 0.23 61.5 55.0 326 3.95 3.98 2.43
1 0.21 59.8 61.0 326 3.89 3.84 2.31
2 0.23 56.9 65.0 327 4.05 4.07 2.31
3 0.29 62.4 58.0 334 4.20 4.23 2.63
4 0.31 63.3 58.0 335 4.34 4.35 2.75
5 0.24 62.8 57.0 336 3.94 3.96 2.48
6 0.24 62.3 57.0 336 3.95 3.98 2.47
7 0.26 61.9 55.0 337 4.07 4.11 2.53
8 0.22 65.1 61.0 337 3.87 3.78 2.49
9 0.23 59.4 61.0 338 4.00 4.05 2.39
10 0.30 64.0 55.0 339 4.25 4.28 2.73
11 0.23 62.8 56.0 340 3.93 3.90 2.46
12 0.22 60.4 61.0 342 3.88 3.84 2.33
13 0.31 62.2 54.0 344 4.35 4.37 2.71
14 0.20 60.2 62.0 345 3.79 3.75 2.27
15 0.32 60.9 58.0 345 4.38 4.42 2.68
16 0.30 62.0 54.0 348 4.31 4.34 2.68
17 0.30 63.4 54.0 351 4.23 4.29 2.70
18 0.30 63.8 56.0 351 4.23 4.26 2.71
19 0.30 62.7 59.0 351 4.21 4.27 2.66
20 0.30 63.3 56.0 351 4.26 4.30 2.71
21 0.23 63.8 55.0 352 3.85 3.92 2.48
22 0.23 61.0 57.0 353 3.94 3.96 2.41
23 0.31 59.4 62.0 353 4.39 4.43 2.62
24 0.31 58.1 62.0 353 4.44 4.47 2.59
25 0.23 60.4 58.0 354 3.97 4.01 2.41
26 0.24 62.5 57.0 355 3.97 3.94 2.47
27 0.30 62.2 57.0 357 4.28 4.30 2.67
28 0.23 60.5 61.0 357 3.96 3.97 2.40
29 0.23 60.9 57.0 357 3.96 3.99 2.42
... ... ... ... ... ... ... ...
53910 0.70 60.5 58.0 2753 5.74 5.77 3.48
53911 0.57 59.8 60.0 2753 5.43 5.38 3.23
53912 0.61 61.8 59.0 2753 5.48 5.40 3.36
53913 0.80 64.2 58.0 2753 5.84 5.81 3.74
53914 0.84 63.7 59.0 2753 5.94 5.90 3.77
53915 0.77 62.1 56.0 2753 5.84 5.86 3.63
53916 0.74 63.1 59.0 2753 5.71 5.74 3.61
53917 0.90 63.2 60.0 2753 6.12 6.09 3.86
53918 0.76 59.3 62.0 2753 5.93 5.85 3.49
53919 0.76 62.2 55.0 2753 5.89 5.87 3.66
53920 0.70 62.4 60.0 2755 5.57 5.61 3.49
53921 0.70 62.8 60.0 2755 5.59 5.65 3.53
53922 0.70 63.1 59.0 2755 5.67 5.58 3.55
53923 0.73 61.3 56.0 2756 5.80 5.84 3.57
53924 0.73 61.6 55.0 2756 5.82 5.84 3.59
53925 0.79 61.6 56.0 2756 5.95 5.97 3.67
53926 0.71 61.9 56.0 2756 5.71 5.73 3.54
53927 0.79 58.1 59.0 2756 6.06 6.13 3.54
53928 0.79 61.4 58.0 2756 6.03 5.96 3.68
53929 0.71 61.4 56.0 2756 5.76 5.73 3.53
53930 0.71 60.5 55.0 2756 5.79 5.74 3.49
53931 0.71 59.8 62.0 2756 5.74 5.73 3.43
53932 0.70 60.5 59.0 2757 5.71 5.76 3.47
53933 0.70 61.2 59.0 2757 5.69 5.72 3.49
53934 0.72 62.7 59.0 2757 5.69 5.73 3.58
53935 0.72 60.8 57.0 2757 5.75 5.76 3.50
53936 0.72 63.1 55.0 2757 5.69 5.75 3.61
53937 0.70 62.8 60.0 2757 5.66 5.68 3.56
53938 0.86 61.0 58.0 2757 6.15 6.12 3.74
53939 0.75 62.2 55.0 2757 5.83 5.87 3.64

[53940 rows x 7 columns]

We'll do a train/test split, and look at a few rows as a sanity check:

In [6]:
from sklearn.model_selection import train_test_split

X = df.drop(df.columns[3], axis=1)
y = df.iloc[:,3:4]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

print(X[:5])

print(y[:5])

carat depth table x y z
0 0.23 61.5 55.0 3.95 3.98 2.43
1 0.21 59.8 61.0 3.89 3.84 2.31
2 0.23 56.9 65.0 4.05 4.07 2.31
3 0.29 62.4 58.0 4.20 4.23 2.63
4 0.31 63.3 58.0 4.34 4.35 2.75
 price
0 326
1 326
2 327
3 334
4 335

Now we'll build a simple feed-forward perceptron network in Keras, and train it for a minute or so, then check our performance

In [9]:
dbutils.library.installPyPI("findspark") 

Out[ 41 ]: True

In [10]:
import tensorflow as tf
import numpy as np

model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(16, input_dim=6, kernel_initializer='normal', activation='relu')) 
model.add(tf.keras.layers.Dense(1, kernel_initializer='normal', activation='linear'))

model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mean_squared_error'])
model.fit(X_train, y_train, epochs=64, batch_size=128, validation_split=0.1, verbose=2)

scores = model.evaluate(X_test, y_test)
print("\nroot %s: %f" % (model.metrics_names[1], np.sqrt(scores[1])))

WARNING:tensorflow:From /local_disk0/pythonVirtualEnvDirs/virtualEnv-eca6c570-7f18-49d4-a2b3-a33b3c579ca4/lib/python3.5/site-packages/tensorflow/python/ops/resource_variable_ops.py:435: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.
WARNING:tensorflow:From /local_disk0/pythonVirtualEnvDirs/virtualEnv-eca6c570-7f18-49d4-a2b3-a33b3c579ca4/lib/python3.5/site-packages/tensorflow/python/keras/utils/losses_utils.py:170: to_float (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.cast instead.
Train on 38836 samples, validate on 4316 samples
WARNING:tensorflow:From /local_disk0/pythonVirtualEnvDirs/virtualEnv-eca6c570-7f18-49d4-a2b3-a33b3c579ca4/lib/python3.5/site-packages/tensorflow/python/ops/math_ops.py:3066: to_int32 (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.cast instead.
Epoch 1/64
 - 1s - loss: 30947533.3662 - mean_squared_error: 30947528.0000 - val_loss: 28328675.5700 - val_mean_squared_error: 28328676.0000
Epoch 2/64
 - 0s - loss: 28189147.9464 - mean_squared_error: 28189148.0000 - val_loss: 24583030.8267 - val_mean_squared_error: 24583032.0000
Epoch 3/64
 - 0s - loss: 23808754.3360 - mean_squared_error: 23808754.0000 - val_loss: 20234934.5922 - val_mean_squared_error: 20234930.0000
Epoch 4/64
 - 1s - loss: 19645973.5975 - mean_squared_error: 19645970.0000 - val_loss: 16879783.8378 - val_mean_squared_error: 16879784.0000
Epoch 5/64
 - 0s - loss: 16871948.0801 - mean_squared_error: 16871950.0000 - val_loss: 15111471.4995 - val_mean_squared_error: 15111471.0000
Epoch 6/64
 - 0s - loss: 15598971.1398 - mean_squared_error: 15598972.0000 - val_loss: 14530729.5885 - val_mean_squared_error: 14530730.0000
Epoch 7/64
 - 0s - loss: 15195743.7679 - mean_squared_error: 15195746.0000 - val_loss: 14404787.3494 - val_mean_squared_error: 14404787.0000
Epoch 8/64
 - 0s - loss: 15081493.2275 - mean_squared_error: 15081488.0000 - val_loss: 14362067.7183 - val_mean_squared_error: 14362068.0000
Epoch 9/64
 - 0s - loss: 15020463.8206 - mean_squared_error: 15020459.0000 - val_loss: 14315722.8304 - val_mean_squared_error: 14315722.0000
Epoch 10/64
 - 0s - loss: 14961055.0147 - mean_squared_error: 14961056.0000 - val_loss: 14257631.8888 - val_mean_squared_error: 14257630.0000
Epoch 11/64
 - 0s - loss: 14895548.7200 - mean_squared_error: 14895543.0000 - val_loss: 14193376.0936 - val_mean_squared_error: 14193376.0000
Epoch 12/64
 - 0s - loss: 14824181.3958 - mean_squared_error: 14824182.0000 - val_loss: 14118125.0454 - val_mean_squared_error: 14118126.0000
Epoch 13/64
 - 0s - loss: 14745834.3618 - mean_squared_error: 14745836.0000 - val_loss: 14046609.8888 - val_mean_squared_error: 14046610.0000
Epoch 14/64
 - 0s - loss: 14657627.3575 - mean_squared_error: 14657627.0000 - val_loss: 13948552.9045 - val_mean_squared_error: 13948555.0000
Epoch 15/64
 - 0s - loss: 14535988.4295 - mean_squared_error: 14535988.0000 - val_loss: 13816274.2424 - val_mean_squared_error: 13816276.0000
Epoch 16/64
 - 0s - loss: 14394183.7933 - mean_squared_error: 14394180.0000 - val_loss: 13678057.5607 - val_mean_squared_error: 13678057.0000
Epoch 17/64
 - 0s - loss: 14241520.3859 - mean_squared_error: 14241520.0000 - val_loss: 13517724.4773 - val_mean_squared_error: 13517724.0000
Epoch 18/64
 - 0s - loss: 14077978.9392 - mean_squared_error: 14077981.0000 - val_loss: 13358610.0982 - val_mean_squared_error: 13358612.0000
Epoch 19/64
 - 0s - loss: 13901552.9719 - mean_squared_error: 13901553.0000 - val_loss: 13184160.8508 - val_mean_squared_error: 13184163.0000
Epoch 20/64
 - 0s - loss: 13710612.8602 - mean_squared_error: 13710609.0000 - val_loss: 12990781.4476 - val_mean_squared_error: 12990781.0000
Epoch 21/64
 - 0s - loss: 13505630.0396 - mean_squared_error: 13505630.0000 - val_loss: 1278809

In [11]:
model.save("/tmp/model")

In [12]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www-us.apache.org/dist/spark/spark-2.4.2/spark-2.4.2-bin-hadoop2.7.tgz
!tar xf spark-2.4.2-bin-hadoop2.7.tgz
!pip install -q findspark

In [13]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.2-bin-hadoop2.7"

In [14]:
import findspark
#findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

Ok, now let's get Spark DataFrame with our test data. In real life, we'd read this directly from S3, HDFS, Kafka, or wherever.

But since this test set is already on the driver (and not very big), we can make a distributed Spark DF from the Pandas DF.

In [16]:
from pyspark.ml.feature import VectorAssembler

testDF = spark.createDataFrame(pd.DataFrame(X_test, columns=["carat", "depth", "table", "x", "y", "z"]))

testDF.show()

+-----+-----+-----+----+----+----+
carat|depth|table| x| y| z|
+-----+-----+-----+----+----+----+
 0.55| 61.6| 55.0|5.33|5.28|3.27|
 0.71| 61.7| 60.0| 5.7|5.74|3.53|
 0.7| 61.5| 57.0|5.68|5.71| 3.5|
 0.33| 59.1| 60.0|4.48|4.52|2.66|
 0.4| 60.4| 57.0|4.76|4.81|2.89|
 0.71| 62.0| 59.0|5.69|5.65|3.52|
 0.5| 61.8| 56.0|5.15|5.11|3.17|
 1.25| 60.6| 60.0|6.92|6.95| 4.2|
 0.46| 61.7| 60.0|4.94|4.98|3.06|
 0.3| 63.4| 57.0|4.31|4.27|2.72|
 0.57| 61.9| 55.0|5.34|5.36|3.31|
 0.43| 61.8| 55.0|4.85|4.89|3.01|
 1.29| 60.8| 55.0|7.03|7.12| 4.3|
 0.78| 62.3| 56.0|5.91|5.88|3.67|
 1.01| 59.8| 59.0|6.55| 6.5| 3.9|
 0.52| 60.9| 58.0|5.22|5.16|3.16|
 0.7| 61.9| 58.0|5.74|5.69|3.54|
 1.02| 57.5| 62.0|6.62| 6.6| 3.8|
 0.55| 62.7| 59.0|5.22|5.25|3.28|
 1.74| 61.6| 59.0| 7.7|7.62|4.72|
+-----+-----+-----+----+----+----+
only showing top 20 rows

When our data shows up, we'll have to reshape it a little bit, and then we can do a regular keras `model.predict()` on it. A  raw prediction looks like this:

In [18]:
model.predict(X_test[:5])

Out[ 60 ]: 
array([[ 2125.22412109],
 [ 3067.08618164],
 [ 3267.25585938],
 [ 732.95611572],
 [ 1155.28955078]], dtype=float32)

That's almost perfect ... but Spark is going to expect a Pandas Series of outputs from the PandasUDF (one for each input)

In [20]:
pd.Series(model.predict(X_test[:5]).flatten())

Out[ 61 ]: 
0 2125.224121
1 3067.086182
2 3267.255859
3 732.956116
4 1155.289551
dtype: float32

Let's try it:

*Note: in real life, don't load the model on each call ... make sure the model is available ahead of time on the executors and just loaded once (e.g. by distributing a module/pyfile)*

In [22]:
! pip install pyarrow

In [23]:
from pyspark.sql.functions import *
from pyspark.sql.functions import PandasUDFType

import numpy as np

@pandas_udf("double", PandasUDFType.SCALAR)
def keras_predict(*v):
  # reshape to records x len(v) columns
  reshaped = np.asarray(v).reshape(len(v), -1).transpose()
  keras_model = tf.keras.models.load_model('/tmp/model')
  return pd.Series(keras_model.predict(reshaped).flatten())    

And now we can `keras_predict` on the columns in the Spark DataFrame:

In [25]:
testDF.select(keras_predict(*testDF.columns)).show()

+-------------------------------------------+
keras_predict(carat, depth, table, x, y, z)|
+-------------------------------------------+
 2125.22412109375|
 3067.086181640625|
 3267.255859375|
 732.9561157226562|
 1155.28955078125|
 3000.066650390625|
 1653.34912109375|
 7366.7314453125|
 1272.41357421875|
 436.80877685546875|
 2251.216552734375|
 1293.30419921875|
 8262.3720703125|
 3983.386474609375|
 6007.90625|
 1710.121337890625|
 3209.865966796875|
 6143.64404296875|
 1691.0284423828125|
 10172.3486328125|
+-------------------------------------------+
only showing top 20 rows

And now ... we've got Spark and all of our Python goodness playing nice!